## Importing libraries

In [1]:
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import FunctionTransformer
from sklearn.semi_supervised import LabelSpreading , LabelPropagation
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score,accuracy_score
from simcse import SimCSE
import re
import string
import nltk 

## Importing dataset

In [2]:
# Loading dataset containing first five categories
data = fetch_20newsgroups(
    subset="train",
    categories=[
        "alt.atheism",
        "comp.graphics",
        "comp.os.ms-windows.misc",
        "comp.sys.ibm.pc.hardware",
        "comp.sys.mac.hardware",
    ],
)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()

2823 documents
5 categories



In [3]:
X, y = data.data, data.target

## Pre-processing text data

In [4]:
# removing emails from the text
def remove_emails(text):
    text = re.sub('\S*@\S*\s?', '', text)
    return text

# remove duplicate spaces and new lines
def remove_spaces(text):
    text = [" ".join(re.split("\s+", word, flags=re.UNICODE)) for word in text]
    return text

# removing punctuations
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text


    
# tokenization
def tokenization(text):
    text = re.split('\W+', text)
    return text

# removing stopwords
def remove_stopwords(text):
    stopword = nltk.corpus.stopwords.words('english')
    text = [word for word in text if word not in stopword]
    return text

# texting Lemmitization
def lemmatizer(text):
    wn = nltk.WordNetLemmatizer()
    text = [wn.lemmatize(word) for word in text]
    return text

In [5]:
def clean_text(text):
    # convert the text to lowercase
    text_lower = text.lower() 
    
    # removing emails from the text
    text_without_emails = remove_emails(text_lower)
    
    # remove duplicate spaces and new lines
    text_without_spaces = remove_spaces(text_without_emails)
    
    # remove punctuation
    text_without_punctuations = remove_punct(text_without_spaces)
    
    # text tokenization
    tokens = tokenization(text_without_punctuations)   
    
    # removing stopwords 
    text_without_stopwords = remove_stopwords(tokens)
    
    # text Lemmitization
    text_clean = lemmatizer(text_without_stopwords)
    
    return " ".join(text_clean)

In [6]:
X = [clean_text(x) for x in X]
X[0]

'brian v hughes subject new apple ergomouse replyto organization dartmouth college hanover nh disclaimer personally really dont care think speak moderator recartscomicsinfo line schizophrenia mean never alone writes anyone know open apple ergomouse adb mouse ii mine life near cat true really pick fur tell look like apple welded shut must tried hard opend mine second take look bottom dial turn open much like older adb mouse used bit harder turn first quite simple open also anyone know installing fpus mac lc iii ive heard people saying fried motherboard lc iii well dont match pin correctly problem close look socket give idea proper orientation chip hades '

## Creating the embeddings model and transforming the data

In [ ]:
# loading SimCSE embeddings model
embeddings_model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")

def encoding_with_embeddings(model,data):
    embeddings = model.encode(data)
    return embeddings
X = encoding_with_embeddings(embeddings_model,X)

/home/abdelrhman/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
 71%|██████████████████████████████▌            | 32/45 [03:46<01:36,  7.46s/it]

## creating the machine learning models

In [ ]:
# Label Propagation Model
lp_model = LabelPropagation()

# Label Spreading Model
ls_model = LabelSpreading()

# function for training and evaluating each model
def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:", sum(1 for x in y_train if x == -1))
    
    # training the model
    clf.fit(X_train, y_train)
    
    # making predictions
    y_pred = clf.predict(X_test)
    print(
        "accuracy score on test set: %0.3f"
        % accuracy_score(y_test, y_pred)
    )
    print("-" * 10)
    print()


## Splitting the dataset

### Spliting into training data and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Creating unlabeled data

In [ ]:
# select a mask of 20% of the train dataset
mask_percentage = 0.2
y_mask = np.random.rand(len(y_train)) < mask_percentage


# set the non-masked subset to be unlabeled
y_train[~y_mask] = -1

print(f"LabelSpreading on {mask_percentage*100}% of the data (rest is unlabeled):")


## Evaluating the models

### Label Propagation Model

In [ ]:
eval_and_print_metrics(lp_model, X_train, y_train, X_test, y_test)

### Label Spreading Model

In [ ]:
eval_and_print_metrics(ls_model, X_train, y_train, X_test, y_test)